## 随机坐标下降

In [1]:
import scipy.sparse as sp
import numpy as np

定义一个凸目标函数：
$$
\min_{x} \frac{1}{2} ||Ax-b||^{2}
$$

定义一个读取数据的函数`Fun()`, 读取参数$A$和$b$, 以及计算目标函数值的`min_f`:

In [2]:
class Fun(object):
    def __init__(self, path_a, path_b):
        self.file_a_path = path_a
        self.file_b_path = path_b
        self.A, self.b = self._get_parameter()
        self.A_T = self.A.transpose()
        self.x_init = sp.eye(self.A.shape[-1], self.b.shape[-1]).tocsr()  # 设置初始解

    def min_f(self, x):
        y = self.A * x - self.b
        return np.linalg.norm(y.toarray(), ord=2)

    def _get_parameter(self):
        self.A = sp.load_npz(self.file_a_path).tocsr()  # shape = (15935, 62061)
        self.b = sp.load_npz(self.file_b_path).tocsr()  # shape = (15935, 1)
        return self.A, self.b

随机坐标下降的迭代公式：

$$
x_{k+1}^{i} = x_{k}^{i} - \eta (a^{i})^{T}(Ax_{k}-b)
$$

因为$A$的维度为`(15935, 62061)`，$b$的维度为`(15935, 1)`。所以$x$的维度为`(62061, 1)`，$x_{k}$的维度为`(62061, 1)`。$Ax_{k}-b$的纬度为`(15935, 1)`。$a^{i}$的维度为`(15935, 1)`, $(a^{i})^{T}(Ax_{k}-b)$的维度`(1, 1)`能够与$x_{k}^{i}$对齐。

In [3]:
class StochasticCoordinateDescent(Fun):
    def __init__(self, path_a, path_b):
        super(StochasticCoordinateDescent, self).__init__(path_a, path_b)

    def stochastic_coordinate_decs(self, eta=0.002, iter_times=1, x_input=None):

        x_output = None
        y_input = self.min_f(x_input)
        for i in range(iter_times):
            index = np.random.randint(0, self.A.shape[1])
            a_i = self.A[:, index]
            x_input[index] = x_input[index] - eta * (a_i.transpose() * (self.A * x_input - self.b)) # 更新 x 的值
            x_output = x_input
            y_output = self.min_f(x_output)
            print("pre_y is {}  and y is {} pre-y {}".format(y_input, y_output, (y_input-y_output)))
            y_input = y_output

        return x_output

In [4]:

if __name__ == "__main__":

    StochasticCD = StochasticCoordinateDescent(path_a='./news20_A.npz', path_b='./news20_b.npz')

    StochasticCD.stochastic_coordinate_decs(eta=0.002, iter_times=20, x_input=StochasticCD.x_init)

    print('')

/Users/hezhiqiang01/Desktop/anaconda/anaconda3/envs/ACG/lib/python3.6/site-packages/scipy/sparse/_index.py:116: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


pre_y is 15.722300259397239  and y is 15.722300185562252 pre-y 7.383498612512085e-08
pre_y is 15.722300185562252  and y is 15.7223000821242 pre-y 1.034380527897838e-07
pre_y is 15.7223000821242  and y is 15.722298722135958 pre-y 1.3599882411341468e-06
pre_y is 15.722298722135958  and y is 15.722298715932318 pre-y 6.203640268154231e-09
pre_y is 15.722298715932318  and y is 15.722298711182471 pre-y 4.749846738150154e-09
pre_y is 15.722298711182471  and y is 15.72229856910537 pre-y 1.4207710208324897e-07
pre_y is 15.72229856910537  and y is 15.722291014928906 pre-y 7.554176463031581e-06
pre_y is 15.722291014928906  and y is 15.722290917826914 pre-y 9.710199222467963e-08
pre_y is 15.722290917826914  and y is 15.72228867713721 pre-y 2.240689704890997e-06
pre_y is 15.72228867713721  and y is 15.722288459949251 pre-y 2.171879582846259e-07
pre_y is 15.722288459949251  and y is 15.722288459717602 pre-y 2.3164936635566846e-10
pre_y is 15.722288459717602  and y is 15.722288438419291 pre-y 2.12983